In [ ]:
import numpy as np
import random

In [ ]:
def initialize_population( num_individuals, num_variables ):
    """
    Khởi tạo quần thể gồm num_individuals cá thể. Mỗi cá thể có num_parameters biến.
    
    Arguments:
    num_individuals -- Số lượng cá thể
    num_variables -- Số lượng biến
    
    Returns:
    pop -- Ma trận (num_individuals, num_variables ) chứa quần thể mới được khởi tạo ngẫu nhiên.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    # pop = np.random.randint(2, size=(num_individuals, num_variables))
    pop = []
    array = [0 if x < num_variables//2 else 1 for x in range(num_variables)]
    pop.append(array)
    array = [x*(-1) + 1 for x in array]
    pop.append(array)
    pop_len = 2
    while pop_len != num_individuals:
      array = [0 if np.random.rand() < 0.5 else 1 for i in range(num_variables) ]
      if(array not in pop):
        pop.append(array)
        array = [x*(-1) + 1 for x in array]
        pop.append(array)
        pop_len+=2
    
    pop = np.array(pop)
    ### DỪNG CODE TẠI ĐÂY ###
    
    return pop

In [ ]:
np.random.seed(20520835)
pop = initialize_population(8,4)
print(pop)

[[0 0 1 1]
 [1 1 0 0]
 [1 0 1 1]
 [0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]
 [1 1 1 0]]


In [ ]:
def onemax( ind ):
    """
    Hàm đánh giá OneMax: Đếm số bit 1 trong chuỗi nhị phân (cá thể ind).
    
    Arguments:
    ind -- Cá thể cần được đánh giá.

    Returns:
    value -- Giá trị của cá thể ind.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ###     
    value = np.sum(ind)
    
    ### DỪNG CODE TẠI ĐÂY ###
    
    return value

In [ ]:
onemax(pop[5,:])

3

In [ ]:
def truncation_selection(pop, pop_fitness, selection_size):
    selected_indices = np.argsort(pop_fitness)[-selection_size:]
    return selected_indices

In [ ]:
def tournament_selection(pop, pop_fitness):
    selected_indices = []
    size = pop_fitness.size
    #round 1
    for i in range (0, size, 4):
      index = i
      max_fitness = pop_fitness[index]
      for j in range (0,4):
        if((pop_fitness[i+j] > max_fitness).any()): index = i+j; max_fitness = pop_fitness[i+j]

      selected_indices.append(index)
    
    #round 2
    for i in range (0, size//2, 2):
      index = i
      max_fitness = pop_fitness[index]
      for j in range (0,2):
        if((pop_fitness[j+i] > max_fitness).any()): index = i+j; max_fitness = pop_fitness[i+j]
        if((pop_fitness[size-1 - j] > max_fitness).any()): index = size-1 - j; max_fitness = pop_fitness[size-1 -j]

      selected_indices.append(index)
    
    selected_indices = np.array(selected_indices)
    return selected_indices

In [ ]:
def crossover( pop ):
    """
    Hàm biến đổi tạo ra các cá thể con.
    
    Arguments:
    pop -- Quàn thể hiện tại.

    Returns:
    offspring -- Quần thể chứa các cá thể con được sinh ra.
    """  
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    num_individuals = len(pop)
    num_parameters = len(pop[0])
    indices = np.arange(num_individuals)
    # Đảo ngẫu nhiên thứ tự các cá thể trong quần thể
    np.random.shuffle(indices)
    offspring = []
    
    for i in range(0, num_individuals, 2):
        idx1 = indices[i]
        idx2 = indices[i+1]
        offspring1 = list(pop[idx1])
        offspring2 = list(pop[idx2])
        
        # Cài đặt phép lai đồng nhất uniform crossover. 
        # Không cần cài đặt đột biến mutation.
        for idx in range(0, num_parameters):
            r = np.random.rand()
            if r < 0.5:
                temp = offspring2[idx] 
                offspring2[idx] = offspring1[idx]
                offspring1[idx] = temp

        offspring.append(offspring1)
        offspring.append(offspring2)


    ### DỪNG CODE TẠI ĐÂY ###
    
    offspring = np.array(offspring)
    return offspring

In [ ]:
def mutation(pop, mutation_prob):
    num_individuals = len(pop)
    num_parameters = len(pop[0])
    for i in range(0, num_individuals):
        for j in range(0, num_parameters):
            r = np.random.rand()
            if r < mutation_prob:
                if pop[i][j] == 0:
                    pop[i][j] = 1
                else:
                    pop[i][j] = 0
    
    return pop

In [ ]:
def genetic_algorithm(num_individuals, num_parameters, num_generations):
    pop = initialize_population(num_individuals, num_parameters)
    pop_fitness = np.array([onemax(ind) for ind in pop])
    print("#Gen 0:")
    print(pop_fitness)

    # selection_size = num_individuals // 2

    for i in range(num_generations):
        # selected_indices = truncation_selection( pop, pop_fitness, selection_size )
        # selected_indices = tournament_selection(pop, pop_fitness)
        # selection_set = pop[selected_indices]
        # selection_fitness = pop_fitness[selected_indices]

        # offspring = crossover(selection_set)
        offspring = crossover(pop)
        offspring = mutation(offspring, 0)
        offspring_fitness = np.array([onemax(ind) for ind in offspring])

        # pop = np.vstack([selection_set, offspring])
        # pop_fitness = np.concatenate((selection_fitness, offspring_fitness))
        pop = np.vstack([pop, offspring])
        pop_fitness = np.concatenate((pop_fitness, offspring_fitness))

        selected_indices = tournament_selection(pop, pop_fitness)
        selection_set = pop[selected_indices]
        selection_fitness = pop_fitness[selected_indices]

        pop = selection_set
        pop_fitness = selection_fitness

        print(f'#Gen {i+1}:')
        print(pop_fitness)
    
    print('#Final result:')
    print(pop)
    print(pop_fitness)


In [ ]:
num_parameters = 8
num_individuals = 12
num_generations = 10
genetic_algorithm(num_individuals, num_parameters, num_generations)

#Gen 0:
[4 4 5 3 4 4 1 7 5 3 6 2]
#Gen 1:
[5 7 6 7 4 6 6 6 6 7 6 6]
#Gen 2:
[7 6 7 7 7 6 7 7 6 6 7 6]
#Gen 3:
[7 7 7 8 8 7 7 7 7 7 7 7]
#Gen 4:
[8 8 7 7 8 8 7 8 8 7 7 7]
#Gen 5:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Gen 6:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Gen 7:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Gen 8:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Gen 9:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Gen 10:
[8 8 8 8 8 8 8 8 8 8 8 8]
#Final result:
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
[8 8 8 8 8 8 8 8 8 8 8 8]


In [ ]:
num_parameters = 16
num_individuals = 24
num_generations = 20
genetic_algorithm(num_individuals, num_parameters, num_generations)

#Gen 0:
[ 8  8  9  7  7  9 11  5  8  8 10  6 10  6  8  8 10  6  7  9 13  3  8  8]
#Gen 1:
[ 9 11 10 10 10 13 11 12  9 13 10  9  9  9  9 11  9 10 10  9 10  9 13  9]
#Gen 2:
[11 13 13 11 10 13 14 14 12 12 13 13 13 13 13 13 13 13 13 13 13 13 13 13]
#Gen 3:
[13 14 13 13 13 13 14 13 13 15 14 14 13 13 13 14 13 13 13 13 13 13 13 13]
#Gen 4:
[14 14 15 14 13 13 13 13 15 14 14 15 15 15 15 15 15 15 15 15 15 15 15 15]
#Gen 5:
[15 13 15 15 15 15 16 15 15 15 15 15 14 15 14 14 15 15 15 15 15 15 15 15]
#Gen 6:
[15 16 15 15 15 15 15 16 15 16 15 15 15 15 15 16 15 15 15 15 15 15 15 15]
#Gen 7:
[16 16 16 16 15 15 16 15 16 15 16 15 16 15 15 16 16 15 15 16 15 15 15 15]
#Gen 8:
[16 16 16 16 16 15 15 16 16 16 16 16 16 16 15 16 16 16 16 16 16 16 15 15]
#Gen 9:
[16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]
#Gen 10:
[16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]
#Gen 11:
[16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]
#Gen 12:
[16 1

In [ ]:
num_parameters = 32
num_individuals = 40
num_generations = 20
genetic_algorithm(num_individuals, num_parameters, num_generations)

#Gen 0:
[16 16 14 18 11 21 16 16 17 15 21 11 20 12 13 19 22 10 16 16 17 15 14 18
 12 20 13 19 13 19 18 14 17 15 16 16 19 13 15 17]
#Gen 1:
[18 21 21 20 22 18 20 19 17 19 21 19 21 16 21 16 19 24 19 17 17 18 21 17
 17 21 20 19 22 17 17 18 20 19 19 18 17 17 19 17]
#Gen 2:
[21 22 21 21 24 21 21 22 20 19 20 19 21 22 21 23 21 19 22 22 22 22 22 22
 22 22 22 22 24 22 22 22 22 22 22 22 22 22 22 22]
#Gen 3:
[22 24 20 23 22 22 22 24 22 22 23 22 23 25 23 22 23 22 22 24 22 22 24 22
 22 22 22 23 22 22 22 22 22 22 24 22 22 22 22 22]
#Gen 4:
[24 24 23 25 24 24 23 22 24 22 24 24 24 23 25 24 24 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25]
#Gen 5:
[25 24 24 25 25 25 25 25 25 25 28 26 25 25 27 25 25 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25]
#Gen 6:
[25 25 28 27 25 25 25 25 25 25 28 27 26 25 28 28 26 26 27 27 27 27 27 27
 27 28 27 27 27 27 27 27 27 27 27 27 27 27 27 27]
#Gen 7:
[28 25 28 28 27 27 28 27 27 27 29 28 28 28 29 28 27 26 27 28 27 28 27 27
 2

In [ ]:
num_parameters = 64
num_individuals = 128
num_generations = 25
genetic_algorithm(num_individuals, num_parameters, num_generations)

#Gen 0:
[32 32 28 36 36 28 31 33 29 35 30 34 31 33 26 38 31 33 31 33 30 34 33 31
 27 37 38 26 39 25 35 29 32 32 41 23 37 27 30 34 27 37 34 30 35 29 28 36
 29 35 42 22 27 37 34 30 33 31 36 28 36 28 27 37 29 35 27 37 37 27 36 28
 31 33 30 34 28 36 33 31 29 35 38 26 35 29 36 28 32 32 30 34 32 32 30 34
 35 29 39 25 36 28 35 29 39 25 28 36 31 33 29 35 32 32 29 35 26 38 28 36
 32 32 35 29 37 27 36 28]
#Gen 1:
[36 36 35 38 33 34 38 39 41 37 37 36 42 37 36 37 37 37 34 36 38 36 34 34
 39 36 39 35 35 38 35 37 38 37 37 35 35 35 40 36 34 35 39 34 35 38 34 33
 36 35 38 36 35 37 34 33 36 37 36 33 37 36 35 33 32 36 36 33 35 34 33 38
 33 33 34 33 37 38 39 35 32 41 37 34 37 34 35 36 35 42 37 34 33 36 36 37
 35 37 37 36 33 34 36 33 35 38 35 36 32 34 32 34 35 39 36 35 39 36 33 35
 32 35 38 36 32 35 37 36]
#Gen 2:
[38 39 41 42 37 38 39 38 38 40 39 38 38 37 37 37 36 38 34 39 41 37 42 37
 37 36 38 34 39 39 38 37 41 39 38 38 42 40 40 42 39 38 41 43 40 37 43 41
 39 36 41 38 40 43 38 40 43 37 40 39 38 42 39 39

In [ ]:
num_parameters = 128
num_individuals = 400
num_generations = 35
genetic_algorithm(num_individuals, num_parameters, num_generations)

#Gen 0:
[64 64 72 56 72 56 67 61 70 58 65 63 58 70 61 67 61 67 63 65 63 65 74 54
 68 60 67 61 62 66 71 57 67 61 58 70 62 66 65 63 61 67 69 59 71 57 61 67
 62 66 71 57 67 61 66 62 67 61 59 69 57 71 61 67 63 65 69 59 55 73 61 67
 62 66 56 72 58 70 64 64 63 65 56 72 69 59 56 72 66 62 69 59 75 53 59 69
 59 69 63 65 64 64 63 65 62 66 66 62 64 64 58 70 61 67 56 72 61 67 70 58
 68 60 57 71 61 67 63 65 65 63 53 75 64 64 57 71 74 54 66 62 67 61 61 67
 57 71 64 64 67 61 60 68 70 58 68 60 65 63 60 68 58 70 76 52 66 62 65 63
 67 61 77 51 58 70 60 68 59 69 57 71 75 53 57 71 64 64 61 67 51 77 63 65
 64 64 58 70 69 59 65 63 74 54 69 59 72 56 62 66 66 62 61 67 63 65 61 67
 63 65 63 65 58 70 65 63 62 66 62 66 64 64 61 67 76 52 70 58 60 68 58 70
 71 57 61 67 68 60 65 63 63 65 58 70 66 62 69 59 66 62 68 60 61 67 65 63
 65 63 63 65 62 66 54 74 78 50 69 59 73 55 63 65 67 61 68 60 61 67 65 63
 68 60 59 69 71 57 68 60 65 63 61 67 58 70 68 60 69 59 55 73 63 65 53 75
 61 67 60 68 64 64 70 58 63 65 51 77 71 57 

In [ ]:
# l          number_of_individuals

# 8                  12

# 16                 24

# 32                 40

# 64                 128

# 128                400